In [26]:
!pip install pymupdf


In [27]:
!pip install pymupdf openpyxl pandas

In [28]:
import fitz  # PyMuPDF
import json
from pathlib import Path

PDF_PATH = "/content/Laskentatiedosto_Kisälli PK-KK 31.1.2025.pdf"  # or MURKIONKATU
OUT_JSON = "kisalli_layout.json"


def pdf_to_layout_json(pdf_path, out_json, start_page=0):
    doc = fitz.open(pdf_path)
    pages_data = []

    for page_index in range(start_page, len(doc)):
        page = doc[page_index]
        page_num = page_index + 1

        # --- text spans ---
        text_dict = page.get_text("dict")
        spans = []
        for block in text_dict["blocks"]:
            for line in block.get("lines", []):
                for span in line.get("spans", []):
                    spans.append({
                        "text": span["text"],
                        "bbox": list(span["bbox"]),  # [x0, y0, x1, y1]
                    })

        # --- vector drawings (lines) ---
        lines = []
        for dr in page.get_drawings():
            for item in dr["items"]:
                if item[0] == "l":            # line segment
                    _, p1, p2 = item
                    lines.append({
                        "p1": [p1[0], p1[1]],
                        "p2": [p2[0], p2[1]],
                    })

        pages_data.append({
            "page_number": page_num,
            "width": page.rect.width,
            "height": page.rect.height,
            "spans": spans,
            "lines": lines,
        })

        print(f"Processed page {page_num}")

    with open(out_json, "w", encoding="utf-8") as f:
        json.dump({"pages": pages_data}, f, ensure_ascii=False, indent=2)

    print(f"Saved layout JSON to {out_json}")


if __name__ == "__main__":
    pdf_to_layout_json(PDF_PATH, OUT_JSON, start_page=1)  # start_page=1 skips title page


Processed page 2
Processed page 3
Processed page 4
Processed page 5
Processed page 6
Processed page 7
Processed page 8
Processed page 9
Processed page 10
Processed page 11
Processed page 12
Processed page 13
Processed page 14
Processed page 15
Processed page 16
Saved layout JSON to kisalli_layout.json


In [29]:
import json
import fitz  # PyMuPDF
from pathlib import Path
import pandas as pd
from openpyxl import Workbook
from openpyxl.drawing.image import Image as XLImage
import hashlib
import math

# ================== PATHS ==================

BASE_DIR = Path("/content")  # <-- change to your base folder
JSON_PATH = BASE_DIR / "/content/kisalli_layout.json"
PDF_PATH  = BASE_DIR / "/content/Laskentatiedosto_Kisälli PK-KK 31.1.2025.pdf"

OUT_IMG_DIR = BASE_DIR / "symboli_groups_symbol_only"
OUT_IMG_DIR.mkdir(exist_ok=True)

OUT_XLSX = BASE_DIR / "suoja_symbol_groups_vector_based.xlsx"

# ================== LOAD JSON ==================

data = json.loads(JSON_PATH.read_text(encoding="utf-8"))


# ================== HELPERS ==================

def find_header_span(spans, text):
    for s in spans:
        if s["text"].strip() == text:
            return s
    return None


def vertical_grid_xs(lines, header_y, min_h=20, tol=0.5):
    """X positions of tall vertical table grid lines crossing the header row."""
    xs = []
    for L in lines:
        x0, y0 = L["p1"]
        x1, y1 = L["p2"]
        dx, dy = x1 - x0, y1 - y0
        if abs(dx) < tol and abs(dy) >= min_h and min(y0, y1) <= header_y <= max(y0, y1):
            xs.append(round(x0, 2))
    return sorted(set(xs))


def column_bounds_for_header(header_span, xs):
    """Return [left, right] x of the column that contains this header span."""
    cx = (header_span["bbox"][0] + header_span["bbox"][2]) / 2
    for a, b in zip(xs, xs[1:]):
        if a < cx < b:
            return a, b
    return xs[0], xs[-1]


def find_suoja_cells(page, suoja_left, suoja_right, y_group_tol=3):
    """
    Find Suoja value *cells* on one page using Suoja column bounds.
    Returns list of {'text','top','bottom'}.
    """
    spans = page["spans"]
    cands = []

    for s in spans:
        txt = s["text"].strip()
        if not txt or txt == "Suoja":
            continue
        if not any(ch.isdigit() for ch in txt):
            continue  # must have a number

        x0, y0, x1, y1 = s["bbox"]
        cx = (x0 + x1) / 2
        if not (suoja_left <= cx <= suoja_right):
            continue

        cy = (y0 + y1) / 2
        cands.append((cy, s))

    if not cands:
        return []

    # group close y’s into one cell
    cands.sort(key=lambda t: t[0])
    groups, current, last_y = [], [], None
    for cy, s in cands:
        if last_y is None or abs(cy - last_y) <= y_group_tol:
            current.append(s)
        else:
            groups.append(current)
            current = [s]
        last_y = cy
    if current:
        groups.append(current)

    cells = []
    for g in groups:
        top = min(s["bbox"][1] for s in g)
        bottom = max(s["bbox"][3] for s in g)
        pieces = sorted(g, key=lambda s: s["bbox"][0])
        text = " ".join(p["text"].strip() for p in pieces).strip()
        cells.append({"text": text, "top": top, "bottom": bottom})

    cells.sort(key=lambda c: (c["top"] + c["bottom"]) / 2)
    return cells


def geometry_hash_for_symbol(lines, rect, grid_xs, round_decimals=1):
    """
    Build a hash from the vector geometry of the symbol inside 'rect'
    (only Symboli column band defined by Suoja cell top/bottom).
    """
    x0, y0, x1, y1 = rect
    width, height = x1 - x0, y1 - y0
    if width <= 0 or height <= 0:
        return None

    segments = []
    grid_round = {round(gx, 1) for gx in grid_xs}

    for L in lines:
        lx0, ly0 = L["p1"]
        lx1, ly1 = L["p2"]
        minx, maxx = min(lx0, lx1), max(lx0, lx1)
        miny, maxy = min(ly0, ly1), max(ly0, ly1)

        # skip segments completely outside this row band
        if maxx < x0 - 0.5 or minx > x1 + 0.5 or maxy < y0 - 0.5 or miny > y1 + 0.5:
            continue

        dx, dy = lx1 - lx0, ly1 - ly0
        length = math.hypot(dx, dy)

        # remove very long vertical grid borders
        if abs(dx) < 0.5 and length > height * 0.9:
            continue

        # remove vertical lines exactly on column borders
        if round(minx, 1) in grid_round and round(maxx, 1) in grid_round and length > height * 0.5:
            continue

        # normalize coordinates to rect origin, round for robustness
        nx0 = round(lx0 - x0, round_decimals)
        ny0 = round(ly0 - y0, round_decimals)
        nx1 = round(lx1 - x0, round_decimals)
        ny1 = round(ly1 - y0, round_decimals)
        p = (nx0, ny0)
        q = (nx1, ny1)
        if q < p:
            p, q = q, p
        segments.append((*p, *q))

    if not segments:
        return None

    segments.sort()
    data = str(segments).encode("utf-8")
    return hashlib.md5(data).hexdigest()


# ================== 1) BUILD ROW DATA (USING SUOJA CELL BAND) ==================

doc = fitz.open(str(PDF_PATH))
rows = []

for page in data["pages"]:
    page_num = page["page_number"]
    spans = page["spans"]
    lines = page["lines"]

    sym_h = find_header_span(spans, "Symboli")
    suo_h = find_header_span(spans, "Suoja")
    if not (sym_h and suo_h):
        continue

    # find Symboli & Suoja column x-bounds from vertical grid lines
    header_y = (sym_h["bbox"][1] + sym_h["bbox"][3]) / 2
    xs = vertical_grid_xs(lines, header_y)

    symbol_left, symbol_right = column_bounds_for_header(sym_h, xs)
    suoja_left, suoja_right = column_bounds_for_header(suo_h, xs)

    # all Suoja cells on this page
    cells = find_suoja_cells(page, suoja_left, suoja_right)
    if not cells:
        continue

    pdf_page = doc[page_num - 1]

    for row_idx, cell in enumerate(cells, start=1):
        # *** vertical band from Suoja cell, not from grid ***
        pad = 5  # you can tweak this if needed
        y0 = max(0, cell["top"] - pad)
        y1 = min(page["height"], cell["bottom"] + pad)

        rect = [symbol_left, y0, symbol_right, y1]

        # geometry-based hash for this row’s symbol
        geom_hash = geometry_hash_for_symbol(lines, rect, xs)

        # also make an image (for Excel)
        pix_rect = fitz.Rect(*rect)
        pix = pdf_page.get_pixmap(clip=pix_rect, dpi=200)
        safe_suoja = cell["text"].replace("/", "-").replace(" ", "")
        img_name = f"p{page_num:02d}_row{row_idx:02d}_S-{safe_suoja}.png"
        img_path = OUT_IMG_DIR / img_name
        pix.save(str(img_path))

        rows.append({
            "page_number": page_num,
            "row_index": row_idx,
            "suoja": cell["text"],
            "symbol_geom_hash": geom_hash,
            "image_file": img_name,
        })

print("Total Suoja rows:", len(rows))


# ================== 2) GROUP BY (SUOJA, SYMBOL_GEOM_HASH) ==================

df = pd.DataFrame(rows)

groups = []
for (suoja_val, ghash), sub in df.groupby(["suoja", "symbol_geom_hash"], dropna=False):
    if ghash is None:
        continue  # (optional) skip rows with no geometry
    count = len(sub)
    rep = sub.iloc[0]  # representative for the image
    groups.append({
        "suoja": suoja_val,
        "instances_with_same_symbol": count,
        "image_file": rep["image_file"],
    })

groups_df = pd.DataFrame(groups)
groups_df.sort_values(["suoja", "instances_with_same_symbol"],
                      ascending=[True, False], inplace=True)


# ================== 3) WRITE EXCEL ==================

wb = Workbook()
ws = wb.active
ws.title = "Suoja-Symbol Groups"

headers = ["Suoja value", "Total instances with same symbol", "Symbol"]
for col_idx, h in enumerate(headers, start=1):
    ws.cell(row=1, column=col_idx, value=h)

for i, row in groups_df.iterrows():
    excel_row = i + 2

    ws.cell(row=excel_row, column=1, value=str(row["suoja"]))
    ws.cell(row=excel_row, column=2, value=int(row["instances_with_same_symbol"]))

    img_path = OUT_IMG_DIR / row["image_file"]
    if img_path.exists():
        img = XLImage(str(img_path))
        img.width = 80
        img.height = 80
        ws.add_image(img, f"C{excel_row}")
        ws.row_dimensions[excel_row].height = 60

ws.column_dimensions["A"].width = 18
ws.column_dimensions["B"].width = 26
ws.column_dimensions["C"].width = 18

wb.save(OUT_XLSX)
print("Excel written to:", OUT_XLSX)


Total Suoja rows: 126
Excel written to: /content/suoja_symbol_groups_vector_based.xlsx


In [23]:
!rm -rf /content/symboli_only_col
print("Directory deleted successfully.")

Directory /content/keskus_symbols2 deleted successfully.
